In [8]:
import numpy as np
from dotenv import load_dotenv
import os

In [9]:
from gigachat import GigaChat

In [10]:
import pandas as pd

In [11]:
load_dotenv()

True

In [12]:
credentials = os.environ.get('CREDENTIALS')

In [13]:
with GigaChat(credentials=credentials, verify_ssl_certs=False) as giga:
    response = giga.chat("Какие факторы влияют на стоимость страховки на дом?")
    print(response.choices[0].message.content)

Стоимость страховки на дом зависит от нескольких факторов, включая тип и размер дома, его местоположение, возраст и состояние здания, наличие систем безопасности и другие факторы. Кроме того, стоимость может различаться в зависимости от страховой компании и условий договора.


In [6]:
def build_promt(claim):
    start = 'Извлеки именованные сущности из текста: '
    return start + claim

In [7]:
text4 = 'Директору ГБОУ «Школа № 1527» Кадыковой Елене Владимировне KadykovaEV@edu.mos.ru , 1527@edu.mos.ru Адрес: 115470, г.Москва, пр-кт Андропова, дом 17, корпус 5. Копия: в Южное окружное управление департамента образования города Москвы Адрес: г.Москва, Высокая улица, 14.        от Родителей учеников 6 «В» класса,) указанных ниже в подписях КОЛЛЕКТИВНАЯ ЖАЛОБА Уважаемая Елена Владимировна! С «19» октября 2020 года, с началом введения дистанционного обучения в системе МЭШ для 6-11 классов в связи с эпидемией COVID-19, наши дети, ученики 6 &quot;В&quot; класса не могут ежедневно подключиться к одному или нескольким урокам.'

In [8]:
with GigaChat(credentials=credentials, verify_ssl_certs=False) as giga:
    response = giga.chat(build_promt(text4))
    print(response.choices[0].message.content)

Именованные сущности в тексте:
— Директор ГБОУ «Школа № 1527» Кадыкова Елена Владимировна
— 1527@edu.mos.ru
— Адрес: 115470, г.Москва, пр-кт Андропова, дом 17, корпус 5
— Южное окружное управление департамента образования города Москвы
— Высокая улица, 14
— Родители учеников 6 «В» класса


In [16]:
df = pd.read_csv('../data/train_dataset_train.csv', sep=';')

In [17]:
df.head()

,Исполнитель,Группа тем,Текст инцидента,Тема
0,Лысьвенский городской округ,Благоустройство,"'Добрый день. Сегодня, 20.08.22, моя мать шла ...",★ Ямы во дворах
1,Министерство социального развития ПК,Социальное обслуживание и защита,"'Пермь г, +79194692145. В Перми с ноября 2021 ...",Оказание гос. соц. помощи
2,Министерство социального развития ПК,Социальное обслуживание и защита,'Добрый день ! Скажите пожалуйста если подовал...,Дети и многодетные семьи
3,Город Пермь,Общественный транспорт,'Каждая из них не о чем. Люди на остановках хо...,Содержание остановок
4,Министерство здравоохранения,Здравоохранение/Медицина,'В Березниках у сына привитого откоронавируса ...,Технические проблемы с записью на прием к врачу


In [13]:
df.Тема.nunique()

195

In [14]:
df['Группа тем'].nunique()

26

In [15]:
df_dic = df[['Группа тем', 'Тема']].drop_duplicates().rename(
    columns={'Группа тем': 'group', 'Тема': 'subject'}).sort_values('group')
df_dic

,group,subject
2260,Безопасность,Беженцы
143,Безопасность,Безопасность общественных пространств
201,Безопасность,Отлов безнадзорных собак и кошек
0,Благоустройство,★ Ямы во дворах
227,Благоустройство,★ Нарушение правил уборки от снега и наледи те...
...,...,...
991,Экология,Выбросы вредных веществ в водоёмы/загрязнение ...
29,Экономика и бизнес,Трудоустройство
63,Экономика и бизнес,Цены и ценообразование
3520,Электроснабжение,★ Повреждение опор ЛЭП


In [50]:
def build_prompt(group, incident):
    subjects = df_dic.loc[df_dic.group == group, 'subject'].to_list()
    prompt = 'Существует набор темы: '
    prompt += ' '.join([' ' + l + ' ; ' for l in subjects])
    prompt += " \n Какая тема из набора подходит для текста:" + f"'{incident}'"
    return prompt

In [51]:
build_prompt(df.iloc[6]['Группа тем'], df.iloc[6]['Текст инцидента'])

"Существует набор темы:  Питание ;   Устройство в ДОУ ;   Проблемы с отоплением детских садов и школ ;   Выплаты стипендий ;   Социальная поддержка учителей ;   Детские оздоровительные лагеря ;   Зарплата учителей ;   Дополнительное образование ;   Безопасность образовательного процесса ;   ЕГЭ, ОГЭ ;   Дистанционное образование ;   Общее впечатление ;   Образовательный процесс ;   Строительство школ, детских садов ;   Кадровые перестановки ;   Нехватка мест в школах ;   ВУЗы и ССУЗы ;   Содержание гос. образовательных организаций ;  \n Какая тема из набора подходит для текста:''Здравствуйте, уточните в какую погоду дети начальной школы должны заниматься весной и осенью на улице ? Какие ограничения по температурному режиму и погодным условиям ?'"

In [52]:
with GigaChat(credentials=credentials, verify_ssl_certs=False) as giga:
    response = giga.chat(build_prompt(df.iloc[6]['Группа тем'], df.iloc[6]['Текст инцидента']))

ResponseError: (URL('https://gigachat.devices.sberbank.ru/api/v1/chat/completions'), 500, b'{"status":500,"message":"Internal Server Error"}\n', Headers([('server', 'nginx'), ('date', 'Fri, 24 Nov 2023 21:09:37 GMT'), ('content-type', 'application/json; charset=utf-8'), ('content-length', '49'), ('connection', 'keep-alive'), ('access-control-allow-credentials', 'true'), ('access-control-allow-headers', 'Origin, X-Requested-With, Content-Type, Accept, Authorization'), ('access-control-allow-methods', 'GET, POST, DELETE, OPTIONS'), ('access-control-allow-origin', 'https://beta.saluteai.sberdevices.ru'), ('x-request-id', '085cd093-576e-4bce-a778-fddbb6d06a01'), ('x-session-id', 'd5e44631-d86a-4ca3-a661-8b22fcdc251c'), ('allow', 'GET, POST'), ('strict-transport-security', 'max-age=31536000; includeSubDomains'), ('allow', 'GET, POST'), ('strict-transport-security', 'max-age=31536000; includeSubDomains')]))

In [50]:
print(response)

choices=[Choices(message=Messages(role='assistant', content='Тема "Образовательный процесс" подходит для данного текста.'), index=0, finish_reason='stop')] created=1700837012 model='GigaChat:latest' usage=Usage(prompt_tokens=366, completion_tokens=15, total_tokens=381) object_='chat.completion'


In [18]:
import re

In [59]:
def detect_subject(group, incident):
    with GigaChat(credentials=credentials, verify_ssl_certs=False) as giga:
        giga.temperature=0 
        response = giga.chat(build_prompt(group, incident))
    text = response.choices[0].message.content
    pattern = r'"([^"]*)"'
    subject =''
    match = re.search(pattern, text)
    if match:
        subject = match.group(1)
    # print(incident)
    # print('>>>>',subject)
    return subject

In [35]:
detect_subject(df.iloc[6]['Группа тем'], df.iloc[6]['Текст инцидента'])

'Безопасность образовательного процесса'

In [75]:
df['subject'] = df[0:1000].apply(lambda row: detect_subject(row['Группа тем'], row['Текст инцидента']), axis=1)

'Добрый день. Сегодня, 20.08.22, моя мать шла по улице Ленина между домами 96 и 94. Фонари не горят, упала в яму, которую не видно. Сильно ударилась, остались синяки, очень больно. Благо шла не одна.<br>Уважаемая Администрация, сделайте с этим что-нибудь, да и не только с этим. Ходить опасно не только взрослым, но и детям. Если бы упал маленький ребёнок, было бы намного хуже. Фото прилагаю. Спасибо!
>>>> Опасность на улицах города
'Пермь г, +79194692145. В Перми с ноября 2021 года не работает социальное такси. Каким образом можно получить льготу по проезду в такси в соц учреждения инвалиду 2гр.пррезд в общественном транспорте не возможен. Да и проездного льготного не представляется
>>>> Социальное такси
'Добрый день ! Скажите пожалуйста если подовала на пособие с 3 до 7 2 декабря , когда можно повторно подать ? вроде за 30 дней можно
>>>> Задержка выплат гражданам
'Каждая из них не о чем. Люди на остановках хотят укрыться от непогоды или слишком погоды. Присесть, поставить сумку. Лавоч

In [77]:
df['subject'] = df[1001:5000].apply(lambda row: detect_subject(row['Группа тем'], row['Текст инцидента']), axis=1)

'Добрый день.<br>Прошу разъяснить вопрос: на каких основаниях законным представителям детей не разрешают проход на территории школ?<br>Родители являются участниками обучающего процесса по закону(ст. 2 п. 31 Федерального закона «Об образовании в РФ»).
>>>> Безопасность образовательного процесса
'здравствуйте,лежу в краевом онкологическом диспансере в радиологии,хочу пожаловаться на зав. отделением.Попросила проводить процедуру в первой половине дня,так как во второй неважно себя чувствую.Так вообще меня из вредности,во вторую смену вечером поставила,чтобы не надоедала ей своим пребыванием здесь.Естественно я очень разнервничалась,может ли зав.отделением доводить больных с онкологией до такого состояния?она для чего тут находится?она меня не слышит,у нее мозг со слуховым восприятием не согласуется.Помогите.
>>>> Отношение персонала к пациентам
'Кто знает ,как работает больница у нас в праздники?????
>>>> Оказание медицинской помощи не в полном объеме/отказ в оказании медицинской помощи
'

In [80]:
df.to_pickle('claim5000.pkz', compression='gzip')    

In [84]:
df

,Исполнитель,Группа тем,Текст инцидента,Тема,subject
0,Лысьвенский городской округ,Благоустройство,"'Добрый день. Сегодня, 20.08.22, моя мать шла ...",★ Ямы во дворах,NaN
1,Министерство социального развития ПК,Социальное обслуживание и защита,"'Пермь г, +79194692145. В Перми с ноября 2021 ...",Оказание гос. соц. помощи,NaN
2,Министерство социального развития ПК,Социальное обслуживание и защита,'Добрый день ! Скажите пожалуйста если подовал...,Дети и многодетные семьи,NaN
3,Город Пермь,Общественный транспорт,'Каждая из них не о чем. Люди на остановках хо...,Содержание остановок,NaN
4,Министерство здравоохранения,Здравоохранение/Медицина,'В Березниках у сына привитого откоронавируса ...,Технические проблемы с записью на прием к врачу,NaN
...,...,...,...,...,...
23123,Губахинский городской округ,ЖКХ,'Город Гремячинск-— ситуация с теплом на улице...,Ненадлежащее качество или отсутствие отопления,NaN
23124,Министерство здравоохранения,Здравоохранение/Медицина,"'Здравствуйте у меня ребёнку 2 месяца , тест н...",Технические проблемы с записью на прием к врачу,NaN
23125,Лысьвенский городской округ,Благоустройство,'А что творится с благоустройством дворов! Воо...,Благоустройство придомовых территорий,NaN
23126,Лысьвенский городской округ,ЖКХ,'Сообщение без текста,Ненадлежащее качество или отсутствие отопления,NaN


In [20]:
df1 = pd.read_pickle('claim5100.pkz', compression='gzip')

In [21]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23128 entries, 0 to 23127
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Исполнитель      23128 non-null  object
 1   Группа тем       23128 non-null  object
 2   Текст инцидента  23128 non-null  object
 3   Тема             23128 non-null  object
 4   subject          3999 non-null   object
dtypes: object(5)
memory usage: 903.6+ KB


In [22]:
from tqdm import tqdm
tqdm.pandas()

In [60]:
df1['subject']=df1[df1['subject'].isna()][0:5000].progress_apply(lambda row: detect_subject(row['Группа тем'], row['Текст инцидента']), axis=1)

100%|██████████| 5000/5000 [1:24:01<00:00,  1.01s/it]


In [61]:
df.to_pickle('claim10000.pkz', compression='gzip')    

In [ ]:
df1['subject']=df1[df1['subject'].isna()][0:10000].progress_apply(lambda row: detect_subject(row['Группа тем'], row['Текст инцидента']), axis=1)

  0%|          | 37/10000 [00:37<2:29:16,  1.11it/s]

In [ ]:
df.to_pickle('claim20000.pkz', compression='gzip')    

In [30]:
def giga_letter(incident, subject, person):
    with GigaChat(credentials=credentials, verify_ssl_certs=False) as giga:
        giga.temperature=0 
        response = giga.chat(build_prompt_letter(incident, subject, person))
    text = response.choices[0].message.content
    # pattern = r'"([^"]*)"'
    # subject =''
    # match = re.search(pattern, text)
    # if match:
    #     subject = match.group(1)
    # print(incident)
    # print('>>>>',subject)
    return text

In [33]:
def build_prompt_letter(incident, subject, person):
    prompt = f" Модератор получил следующий текст:  '{incident}'"
    prompt += f" Модератор определил тему письма:  '{subject}.'"
    prompt += f" Модератор определил организацию исполнителя:  '{person}.'"
    prompt += f" Напиши запрос в адрес организации  от имени модератора о проблеме из текста. В запрос включи: кратко о проблеме, требование плана устранения проблемы. проблема из текста должна быть описана от третьего лица."
    return prompt

In [34]:
idx=0
incident= df['Текст инцидента'][idx]
subject=df['Тема'][idx]
person=df['Исполнитель'][idx]
build_prompt_letter(incident, subject, person)

" Модератор получил следующий текст:  ''Добрый день. Сегодня, 20.08.22, моя мать шла по улице Ленина между домами 96 и 94. Фонари не горят, упала в яму, которую не видно. Сильно ударилась, остались синяки, очень больно. Благо шла не одна.<br>Уважаемая Администрация, сделайте с этим что-нибудь, да и не только с этим. Ходить опасно не только взрослым, но и детям. Если бы упал маленький ребёнок, было бы намного хуже. Фото прилагаю. Спасибо!' Модератор определил тему письма:  '★ Ямы во дворах.' Модератор определил организацию исполнителя:  'Лысьвенский городской округ.' Напиши запрос в адрес организации  от имени модератора о проблеме из текста. В запрос включи: кратко о проблеме, требование плана устранения проблемы. проблема из текста должна быть описана от третьего лица."

In [35]:
giga_letter(incident, subject, person)

'Уважаемые представители Лысьвенского городского округа!\n\nСогласно предоставленной информации, на улице Ленина между домами 96 и 94 имеется яма, которая не обозначена соответствующим образом. Это создает опасность для прохожих, включая детей. В частности, 20 августа текущего года моя мать, следуя по указанному маршруту, упала в эту яму и получила сильные ушибы. Мы требуем принять меры по устранению данной проблемы и обеспечить безопасность жителей города.\n\nПросим вас предоставить план действий по устранению проблемы, включая обозначение всех ям и другие меры безопасности на улицах города. Также просим обратить внимание на состояние детских площадок и обеспечить их безопасность.\n\nМы надеемся на вашу оперативную реакцию и решение данной проблемы в кратчайшие сроки.\n\nС уважением,\n[Ваше имя]'